In [ ]:
from datetime import datetime
from pathlib import Path

# 初期設定値

wav_dir = Path(r"D:\Diana\新千歳空港\WAVE") ## WAVEファイルのパス
stid = "CH53"   ## 測定局ID
calibrated_time = datetime(year=2022,month=12,day=1,hour=0,minute=0,second=0)   ## 校正完了時間
sample_size = 300   ## サンプルサイズ
center_freqs = [250,500,1000,2000,4000]
tap = 1024
percentile = 90
mean_time_sec = 0.1
target_freq = "2000"


In [ ]:
# WAVファイルを集める

from datetime import timedelta
import learning_test as ln

time = calibrated_time
wav_file_list = []
while len(wav_file_list) < sample_size:
    event_time, path = ln.get_event_time(stid, wav_dir, time)
    if not event_time:
        print("失敗")
        break
    wav_file_list.append(path)
    time = event_time + timedelta(seconds = 1)
print(f"{calibrated_time}～{event_time}")

In [ ]:
# 実音データから異常検知パラメータを求める

import pandas as pd
import learning_test as ln

columns = list(map(str,center_freqs))
dataset = pd.DataFrame(columns=columns)
for file in wav_file_list:
    fs, signal = ln.wav_load(file)
    oct_freq_masks = ln.make_oct_masks(center_freqs, tap, fs)
    data_list = ln.detect_diff_data(signal, tap, oct_freq_masks, fs, mean_time_sec, percentile)
    dataset = pd.concat([dataset,pd.DataFrame([data_list], columns=columns)], axis=0)

In [ ]:
# 求めたパラメータのデータ分布を確認する

from seaborn_analyzer import hist

n,std,se,mean,sk,ku = ln.get_statics(list(dataset[target_freq]))
print(f"サンプルサイズ={n} 平均={mean} 標準偏差={std} 誤差={se} 歪度={sk} 尖度={ku}")

hist.plot_normality(dataset, x=target_freq, rounddigit=2)
all_params, all_scores = hist.fit_dist(dataset, x=target_freq, dist=['norm', 'gamma', 'lognorm','cauchy','t'])
df_scores = pd.DataFrame(all_scores).T
df_scores

In [ ]:
# 外れ値を除外する smirnovgrubbs検定

o_dataset = {f'{freq}': list() for freq in oct_freq_masks.keys()}
for freq in oct_freq_masks.keys():
    o_dataset[freq], out = ln.smirnov_grubbs(list(dataset[freq]), 0.05)
    print(out)


In [ ]:
import learning_test as ln

# 外れ値除外後のデータ分布を確認する

n,std,se,mean,sk,ku = ln.get_statics(o_dataset[target_freq])
print(f"サンプルサイズ={n} 平均={mean} 標準偏差={std} 誤差={se} 歪度={sk} 尖度={ku}")

o_df = pd.DataFrame(o_dataset[target_freq],copy=True)
hist.plot_normality(o_df, x=0, rounddigit=2)
all_params, all_scores = hist.fit_dist(o_df, x=0, dist=['norm', 'gamma', 'lognorm','cauchy','t'])
df_scores = pd.DataFrame(all_scores).T
df_scores

In [ ]:
# データを正規分布に近づける

from sklearn.preprocessing import PowerTransformer
import itertools

pt = PowerTransformer(method="yeo-johnson",standardize=False,copy=True)

t_dataset = {f'{freq}': list() for freq in oct_freq_masks.keys()}
for freq in oct_freq_masks.keys():

    t_df = pd.DataFrame(o_dataset[freq],copy=True)
    pt.fit(t_df[:])
    t_df[:] = pt.transform(t_df[:])

    t_dataset[freq] = list(itertools.chain.from_iterable(t_df[:].values.tolist()))


In [ ]:
from seaborn_analyzer import hist

# 変換後のデータ分布を確認する

n,std,se,mean,sk,ku = ln.get_statics(t_dataset[target_freq])
print(f"サンプルサイズ={n} 平均={mean} 標準偏差={std} 誤差={se} 歪度={sk} 尖度={ku}")

t_df = pd.DataFrame(t_dataset[target_freq],copy=True)
hist.plot_normality(t_df, x=0, rounddigit=2)
all_params, all_scores = hist.fit_dist(t_df, x=0, dist=['norm', 'gamma', 'lognorm','cauchy','t'])
df_scores = pd.DataFrame(all_scores).T
df_scores

In [ ]:
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt

# データセットの平均値信頼区間を求めてグラフに描画する

t_n = len(t_dataset[target_freq])
t_dof = t_n-1
t_mean = np.mean(o_dataset[target_freq])
t_scale = np.std(o_dataset[target_freq],ddof=1)/np.sqrt(t_n)
t_data = stats.t(loc=t_mean, scale=t_scale, df=t_dof)
bottom, upper = t_data.interval(alpha = 0.95)

print(bottom, upper)

fig = plt.figure(figsize=(5, 5))
plt.hist(t_dataset[target_freq], ec = 'white',bins = 20)
plt.axvline(t_mean, color='orange', linestyle='dashed', linewidth=1)
plt.axvspan(bottom, upper, color="lightgreen",alpha = 0.3)
plt.ylabel("count")
plt.legend([f"Dataset mean = {str(f'{t_mean:.3g}')}",
            f"mean interval = {bottom:.3g} to {upper:.3g}"],
            loc='upper left')
